In [1]:
import pandas as pd

In [2]:
a1 = pd.read_csv("data/my_submit_2.csv")

In [6]:
a2 = pd.read_csv("data/my_submit35.csv")

In [7]:
a3 = pd.read_csv("data/my_submit5.csv")

# Merge answers

In [10]:
merged_df = a1.merge(a2, on='problem_id', suffixes=('_a1', '_a2'))
merged_df = merged_df.merge(a3, on='problem_id', suffixes=('', '_a3'))

In [14]:
merged_df = merged_df.rename(columns={'answer': 'answer_a3'})

# Define columns

In [16]:
merged_df['has_different_answers'] = (
    (merged_df['answer_a1'] != merged_df['answer_a2']) |
    (merged_df['answer_a1'] != merged_df['answer_a3']) |
    (merged_df['answer_a2'] != merged_df['answer_a3'])
)

merged_df['has_two_of_three_answer'] = (
    ((merged_df['answer_a1'] == merged_df['answer_a2']) & (merged_df['answer_a1'] != merged_df['answer_a3'])) |
    ((merged_df['answer_a1'] == merged_df['answer_a3']) & (merged_df['answer_a1'] != merged_df['answer_a2'])) |
    ((merged_df['answer_a2'] == merged_df['answer_a3']) & (merged_df['answer_a2'] != merged_df['answer_a1']))
)

In [17]:
merged_df

,problem_id,answer_a1,answer_a2,answer_a3,has_different_answers,has_two_of_three_answer
0,11919,12.0,12.0,12.0,False,False
1,8513,11285.0,11285.0,11285.0,False,False
2,7887,4.0,4.0,4.0,False,False
3,5272,6.0,6.0,6.0,False,False
4,8295,13.0,13.0,13.0,False,False
...,...,...,...,...,...,...
95,3519,341.0,134.0,25.0,True,False
96,7934,12.0,12.0,12.0,False,False
97,9390,217000.0,217000.0,217000.0,False,False
98,7137,22.0,22.0,22.0,False,False


## Statistics

In [23]:
merged_df.describe(include="bool")

,has_different_answers,has_two_of_three_answer
count,100,100
unique,2,2
top,False,False
freq,86,90


In [24]:
merged_df.loc[merged_df.has_different_answers == True, ]

,problem_id,answer_a1,answer_a2,answer_a3,has_different_answers,has_two_of_three_answer
8,6116,1224.0,1182.0,4224.0,True,False
9,4720,3.0,3.0,23.0,True,True
10,12122,40.0,40.0,25.0,True,True
31,5126,2134.0,2.0,2134.0,True,True
37,4849,2341.0,2.0,2341.0,True,True
42,10411,220200.0,222000.0,222200.0,True,False
51,9695,4321.0,4.0,4321.0,True,True
53,8458,323.0,323.2,323.0,True,True
56,6101,1365.0,2235.0,1365.0,True,True
63,12184,2358.0,2358.0,2678.0,True,True


# Consensus answer

In [25]:

def get_consensus_answer(row):
    answers = [row['answer_a1'], row['answer_a2'], row['answer_a3']]
    
    # Если все ответы одинаковые
    if len(set(answers)) == 1:
        return answers[0]
    
    # Если два из трех ответов одинаковые
    for answer in answers:
        if answers.count(answer) == 2:
            return answer
    
    # Если все ответы разные, возвращаем ответ первого
    return answers[0]

# Применяем функцию к DataFrame
merged_df['consensus_answer'] = merged_df.apply(get_consensus_answer, axis=1)

# Submit

In [26]:
def make_submit(answers: pd.Series, filename="data/my_submit.csv"):
    answer_df = answers.reset_index()
    answer_df.columns = ["problem_id", "answer"]
    answer_df.to_csv(filename, index=False)
    

In [28]:
merged_df.set_index('problem_id', inplace=True)

In [30]:
make_submit(merged_df.consensus_answer, filename="data/my_submit_6_consensus.csv")